<a href="https://colab.research.google.com/github/mharoun20/DGE-PTB/blob/main/DGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
---
title: "Microarray Analysis"
output: html_document
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```

# MFC Super Duper Coding Stuff

##  Data Collection



```{}
# specify the path on your computer where the folder that contains the CEL-files is located
celpath = "/setting up/" #my computer's location
```



```{}
# import CEL files containing raw probe-level data into an R AffyBatch object 
data = ReadAffy(celfile.path=celpath)

```

```{}
#retrive the raw expression levels (intensities) of the genes and look at the first few rows
expr = exprs(data)
expr[1:5,]
``` 


```{}
#retrive the samples' phenotypic annotations and store it in the dataframe ph
ph = data@phenoData
ph

``` 

```{}
#check if the samples have phenotypic annotations by exploring the data in the dataframe
ph@data

#apparently, there is no annotations (that is okay) 
``` 


```{}
#Giving the samples informative names in the dataframe
ph@data[ ,1] = c("23wks_control","23.5wks_control","23.6wks_control","24wks_control", "24.7wks_treatment","25.4wks_treatment","25.6wks_treatmet","27.1wks_treatment")
ph@data
``` 

```{}
#MA Plot: asses the extent the variability in expression with regard to  expression level 
#if more variation on high expression values, then we need normalization'

for (i in 1:8)
{
MA_plot = paste("MAplot",i,".jpg",sep="")
jpeg(MA_plot)
MAplot(data,which=i)
dev.off()
}
``` 

```{}
#display MA plots
jj <- readJPEG(MA_plot,native=TRUE)
plot(0:1,0:1,type="n",ann=FALSE,axes=FALSE)
rasterImage(jj,0,0,1,1)

#displays the last sample only, need to edit to disply all

#the plot shows normalization is so much needed
``` 

```{}
data.rma = rma(data)
data.matrix = exprs(data.rma)

#save the data to an output file, data will be log2 transformed and normalized)
write.exprs(data.normalized,file="datanormalization.txt")
``` 

```{}
#check the quality of normalization by comparing raw and normalized data using MA plots

for (i in 1:8)
{
name = paste("MAplotnorm",i,".jpg",sep="")
jpeg(name)
MAplot(data.rma,which=i)
dev.off()
}

``` 

```{}
#step 1: tell limma which samples are control and which are treatment
#by adding a column with sample annotation we call "source" 
ph@data[ ,2] = c("control","control","control","control", "treatment","treatment","treatment","treatment")
colnames(ph@data)[2]="source"
ph@data

``` 

```{}
#step 2: group the control samples together and the treatment samples together
groups = ph@data$source
f = factor(groups,levels=c("control","treatment"))
f
``` 

```{}
#step 3: create a design matrix, a matrix of values of the grouping variable, to run the ANOVA test by limma
design = model.matrix(~ 0 + f)
colnames(design) = c("control","treatment")
``` 

```{}
#Step 4: calculate the mean expression levels using the lmFit() method
data.fit = lmFit(data.matrix,design)
data.fit$coefficients[1:10,]


``` 

```{}
contrast.matrix = makeContrasts(control-treatment,levels=design)
data.fit.con = contrasts.fit(data.fit,contrast.matrix)
``` 

```{}
data.fit.eb = eBayes(data.fit.con)
names(data.fit.eb)
data.fit.eb$coefficients[1:10,]

``` 

```{}
#volcano plot
name2 = "Volcano.jpg"
jpeg(name2)
volcanoplot(data.fit.eb,coef=1,highlight=10)
dev.off()

``` 

```{}
#multiple testing
options(digits=2)
tab = topTable(data.fit.eb,coef=1,number=200,adjust.method="none")
``` 

```{}
top_genes = tab[tab[, "adj.P.Val"] < 0.001, ]
dim(top_genes)
``` 

```{}
topups = top_genes[top_genes[, "logFC"] > 1, ]
dim(topups)
topdowns = top_genes[top_genes[, "logFC"] < -1, ]
dim(topdowns)
``` 


```{}
#create ids
IDs.up = rownames(topups)
IDs.down = rownames(topdowns)
``` 



```{}
#save it
write.table(IDs.up,row.names=FALSE,col.names=FALSE,quote=FALSE,file="d:upIDs_test.txt")
write.table(IDs.down,row.names=FALSE,col.names=FALSE,quote=FALSE,file="d:downIDs_test.txt")
``` 


